In [1]:
!pip install -q transformers accelerate langchain langchain-community faiss-cpu sentence-transformers pymupdf SpeechRecognition google-generativeai langdetect deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 32.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 72.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 47.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 83.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 58.2 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━

In [2]:
# import os
# import pickle
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from langchain_community.llms import HuggingFacePipeline
# from langchain.vectorstores import FAISS
# from langchain.document_loaders import PyMuPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.chains import RetrievalQA

# # Paths
# #DATA_FOLDER = "/kaggle/input/agrollm-data/agrollm_dataset"
# DATA_FOLDER = "/kaggle/input/agrollm-2/dataset/dataset"
# INDEX_DIR = "/kaggle/working/AgroRAG/faiss_index"
# CHUNKS_PATH = "/kaggle/working/AgroRAG/chunks.pkl"

# # 1. Load PDFs from subfolders
# def load_documents_from_folders(root_dir):
#     all_docs = []
#     for subdir, _, files in os.walk(root_dir):
#         for file in files:
#             if file.endswith(".pdf"):
#                 path = os.path.join(subdir, file)
#                 print(f"Loading {path}...")
#                 loader = PyMuPDFLoader(path)
#                 docs = loader.load()
#                 for doc in docs:
#                     doc.metadata['source'] = path
#                     doc.metadata['topic'] = os.path.basename(subdir)
#                 all_docs.extend(docs)
#     return all_docs

# # 2. Split text into chunks
# def split_documents(documents, chunk_size=500, chunk_overlap=50):
#     splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#     return splitter.split_documents(documents)

# # 3. Embed and store
# def build_or_load_faiss_index(chunks, index_path=INDEX_DIR):
#     embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#     if os.path.exists(os.path.join(index_path, "index.faiss")):
#         print("Loading existing FAISS index...")
#         return FAISS.load_local(index_path, embedder)
#     print("Creating new FAISS index...")
#     vectordb = FAISS.from_documents(chunks, embedder)
#     os.makedirs(index_path, exist_ok=True)
#     vectordb.save_local(index_path)
#     return vectordb

# # 4. Load Mistral-7B-Instruct for Local RAG
# def load_local_llm():
#     model_id = "mistralai/Mistral-7B-Instruct-v0.2"
#     tokenizer = AutoTokenizer.from_pretrained(model_id)
#     model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
#     pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0.7)
#     return HuggingFacePipeline(pipeline=pipe)

# # 5. RAG QA Chain with Local Model
# def run_qa_chain(vectorstore, question):
#     llm = load_local_llm()
#     retriever = vectorstore.as_retriever(search_type="similarity", k=4)
#     qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
#     return qa.run(question)

# # Main Pipeline
# if __name__ == "__main__":
#     if os.path.exists(CHUNKS_PATH):
#         print("✅ Loading pre-saved chunks...")
#         with open(CHUNKS_PATH, 'rb') as f:
#             chunks = pickle.load(f)
#     else:
#         print("📄 Loading and chunking documents...")
#         docs = load_documents_from_folders(DATA_FOLDER)
#         chunks = split_documents(docs)
#         os.makedirs(os.path.dirname(CHUNKS_PATH), exist_ok=True)
#         with open(CHUNKS_PATH, 'wb') as f:
#             pickle.dump(chunks, f)

#     print("🧠 Building or loading FAISS index...")
#     db = build_or_load_faiss_index(chunks)

#     print("\n✅ Chunking and indexing complete. You can now use the inference script to ask questions.")

In [ ]:
import os
import pickle
import torch
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from huggingface_hub import login
import speech_recognition as sr
import numpy as np
from PIL import Image
import gc
import wave
import tempfile
import google.generativeai as genai
import base64
import io
import random
import time
from datetime import datetime, timedelta
import re
from langdetect import detect
from deep_translator import GoogleTranslator
import uuid
import threading
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
import queue
import asyncio
import aiohttp
import requests
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import json

# Configure Gemini API
GEMINI_API_KEY = "REPLACE_WITH_YOUR_GEMINI_API_KEY"
genai.configure(api_key=GEMINI_API_KEY)

# Weather API Configuration (using Open-Meteo - free, no key required)
WEATHER_API_BASE = "https://api.open-meteo.com/v1/forecast"

# CHANGED: Alternative Soil API - using World Soil Information Service (free, no auth)
# Alternative option: Use Gemini AI to estimate soil properties based on location
SOIL_ESTIMATION_MODE = "gemini"  # Options: "gemini" or "static"

# Crop Price API Configuration (using mock data for now - can integrate real API)
CROP_PRICE_API_BASE = "REPLACE_WITH_YOUR_CROP_PRICE_API_BASE"  # Example Indian govt API

# Login to Hugging Face
login("REPLACE_WITH_YOUR_HUGGING_FACE_TOKEN")

# OPTIMIZATION: Set optimal GPU settings
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# Global variables
qa_chain = None
your_llm_pipeline = None
recognizer = sr.Recognizer()
gemini_model = None
INDEX_PATH = "/kaggle/working/AgroRAG/faiss_index"
geolocator = Nominatim(user_agent="agrorag_assistant")

# CONCURRENCY SETTINGS
MAX_CONCURRENT_REQUESTS = 5
REQUEST_QUEUE_SIZE = 20
executor = ThreadPoolExecutor(max_workers=MAX_CONCURRENT_REQUESTS, thread_name_prefix="AgroRAG")
request_semaphore = threading.Semaphore(MAX_CONCURRENT_REQUESTS)

# SUPPORTED LANGUAGES
SUPPORTED_LANGUAGES = {
    'en': 'English', 'hi': 'Hindi', 'bn': 'Bengali', 'te': 'Telugu',
    'mr': 'Marathi', 'ta': 'Tamil', 'gu': 'Gujarati', 'kn': 'Kannada',
    'ml': 'Malayalam', 'pa': 'Punjabi', 'or': 'Odia', 'as': 'Assamese', 'ur': 'Urdu'
}

# DYNAMIC DATA CACHE (thread-safe)
class DataCache:
    def __init__(self, ttl_minutes=30):
        self.cache = {}
        self.lock = threading.Lock()
        self.ttl = timedelta(minutes=ttl_minutes)
    
    def get(self, key):
        with self.lock:
            if key in self.cache:
                data, timestamp = self.cache[key]
                if datetime.now() - timestamp < self.ttl:
                    return data
                del self.cache[key]
            return None
    
    def set(self, key, value):
        with self.lock:
            self.cache[key] = (value, datetime.now())
    
    def clear_old(self):
        with self.lock:
            current_time = datetime.now()
            expired = [k for k, (_, ts) in self.cache.items() if current_time - ts >= self.ttl]
            for k in expired:
                del self.cache[k]

weather_cache = DataCache(ttl_minutes=30)
soil_cache = DataCache(ttl_minutes=60)
crop_cache = DataCache(ttl_minutes=120)
price_cache = DataCache(ttl_minutes=1440)  # 24 hours for price data

class SessionManager:
    def __init__(self, max_sessions=100, max_history_per_session=10):
        self.session_histories = {}
        self.session_lock = threading.Lock()
        self.max_sessions = max_sessions
        self.max_history = max_history_per_session
        self.session_timestamps = {}
        self.active_requests = defaultdict(int)
        self.location_cache = {}
    
    def get_session_id(self, request: gr.Request = None, provided_session_id: str = None):
        if provided_session_id and provided_session_id.strip():
            return f"api_{provided_session_id.strip()}"
        if request and hasattr(request, 'session_hash') and request.session_hash:
            return f"web_{request.session_hash}"
        return f"temp_{str(uuid.uuid4())[:8]}"
    
    def fetch_session_id(self, request: gr.Request = None):
        if request and hasattr(request, 'session_hash') and request.session_hash:
            session_id = f"web_{request.session_hash}"
            return {"status": "success", "session_id": session_id, "display_id": session_id[-8:], 
                   "type": "web_ui", "timestamp": datetime.now().isoformat()}
        new_id = f"api_{str(uuid.uuid4())[:8]}"
        return {"status": "success", "session_id": new_id, "display_id": new_id[-8:], 
               "type": "generated", "timestamp": datetime.now().isoformat()}
    
    def get_history(self, session_id):
        with self.session_lock:
            return self.session_histories.get(session_id, [])
    
    def add_to_history(self, session_id, user_input, bot_response, image_context="", 
                      original_lang="en", is_detailed=False, question_type="new_question", 
                      location=None, dynamic_data=None):
        with self.session_lock:
            if session_id not in self.session_histories:
                self.session_histories[session_id] = []
            
            exchange = {
                "timestamp": datetime.now().isoformat(),
                "user_input": user_input,
                "bot_response": bot_response,
                "image_context": image_context,
                "language": original_lang,
                "is_detailed": is_detailed,
                "question_type": question_type,
                "location": location,
                "dynamic_data": dynamic_data
            }
            
            self.session_histories[session_id].append(exchange)
            self.session_timestamps[session_id] = datetime.now()
            
            if len(self.session_histories[session_id]) > self.max_history:
                self.session_histories[session_id] = self.session_histories[session_id][-self.max_history:]
            
            self._cleanup_old_sessions()
    
    def _cleanup_old_sessions(self):
        if len(self.session_histories) > self.max_sessions:
            sorted_sessions = sorted(self.session_timestamps.items(), key=lambda x: x[1], reverse=True)
            sessions_to_keep = [sid for sid, _ in sorted_sessions[:self.max_sessions]]
            sessions_to_remove = set(self.session_histories.keys()) - set(sessions_to_keep)
            for sid in sessions_to_remove:
                if sid in self.session_histories:
                    del self.session_histories[sid]
                if sid in self.session_timestamps:
                    del self.session_timestamps[sid]
    
    def clear_session_history(self, session_id):
        with self.session_lock:
            if session_id in self.session_histories:
                del self.session_histories[session_id]
            if session_id in self.session_timestamps:
                del self.session_timestamps[session_id]
    
    def increment_active_requests(self, session_id):
        with self.session_lock:
            self.active_requests[session_id] += 1
    
    def decrement_active_requests(self, session_id):
        with self.session_lock:
            self.active_requests[session_id] = max(0, self.active_requests[session_id] - 1)
    
    def get_session_stats(self):
        with self.session_lock:
            return {
                "total_sessions": len(self.session_histories),
                "total_exchanges": sum(len(h) for h in self.session_histories.values()),
                "active_sessions": len([s for s, t in self.session_timestamps.items() 
                                      if (datetime.now() - t).seconds < 3600]),
                "active_requests": sum(self.active_requests.values()),
                "max_sessions": self.max_sessions,
                "max_history": self.max_history
            }
    
    def set_location(self, session_id, location):
        with self.session_lock:
            self.location_cache[session_id] = location
    
    def get_location(self, session_id):
        with self.session_lock:
            return self.location_cache.get(session_id)

session_manager = SessionManager()

# ===== DYNAMIC DATA FETCHING FUNCTIONS =====

def geocode_location(location_query):
    """Convert location query to coordinates using Geopy"""
    try:
        location = geolocator.geocode(location_query, timeout=10)
        if location:
            return {
                "name": location.address,
                "latitude": location.latitude,
                "longitude": location.longitude,
                "success": True
            }
        return {"success": False, "error": "Location not found"}
    except GeocoderTimedOut:
        return {"success": False, "error": "Geocoding timeout"}
    except Exception as e:
        return {"success": False, "error": str(e)}

def fetch_weather_data(latitude, longitude, days=15):
    """UPDATED: Fetch 15-day weather forecast from Open-Meteo API"""
    cache_key = f"weather_{latitude}_{longitude}_{days}"
    cached = weather_cache.get(cache_key)
    if cached:
        print(f"Using cached weather data")
        return cached
    
    try:
        params = {
            'latitude': latitude,
            'longitude': longitude,
            'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,'
                    'precipitation_probability_max,windspeed_10m_max,relative_humidity_2m_mean',
            'forecast_days': days,  # Changed to 15 days
            'timezone': 'auto'
        }
        
        response = requests.get(WEATHER_API_BASE, params=params, timeout=15)
        response.raise_for_status()
        data = response.json()
        
        weather_data = {
            "success": True,
            "location": {"lat": latitude, "lon": longitude},
            "daily_forecast": []
        }
        
        daily = data.get('daily', {})
        dates = daily.get('time', [])
        
        for i in range(len(dates)):
            weather_data["daily_forecast"].append({
                "date": dates[i],
                "temp_max": daily.get('temperature_2m_max', [])[i],
                "temp_min": daily.get('temperature_2m_min', [])[i],
                "precipitation_mm": daily.get('precipitation_sum', [])[i],
                "rain_mm": daily.get('rain_sum', [])[i],
                "precipitation_prob": daily.get('precipitation_probability_max', [])[i],
                "windspeed_kmh": daily.get('windspeed_10m_max', [])[i],
                "humidity_percent": daily.get('relative_humidity_2m_mean', [])[i]
            })
        
        weather_cache.set(cache_key, weather_data)
        print(f"Fetched fresh 15-day weather data for {latitude}, {longitude}")
        return weather_data
        
    except Exception as e:
        print(f"Weather API error: {e}")
        return {"success": False, "error": str(e)}

def fetch_soil_data_gemini(latitude, longitude, location_name):
    """NEW: Use Gemini AI to estimate soil properties based on location"""
    cache_key = f"soil_gemini_{latitude}_{longitude}"
    cached = soil_cache.get(cache_key)
    if cached:
        print(f"Using cached soil data (Gemini)")
        return cached
    
    try:
        prompt = f"""Provide typical soil characteristics for agricultural land near coordinates {latitude}, {longitude} ({location_name}).

Return ONLY valid JSON with this exact structure:
{{
    "ph": 6.5,
    "clay_percent": 25,
    "sand_percent": 40,
    "silt_percent": 35,
    "nitrogen_g_kg": 1.5,
    "organic_carbon_g_kg": 12,
    "soil_type": "loam",
    "drainage": "good",
    "fertility": "medium"
}}

Base estimates on typical soil patterns for this region. Use realistic agricultural values."""

        response = gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.1,
                max_output_tokens=300
            )
        )
        
        if response and response.text:
            text = response.text.strip()
            text = re.sub(r'```json\s*', '', text)
            text = re.sub(r'```\s*$', '', text)
            
            soil_data = json.loads(text)
            soil_data['success'] = True
            soil_data['location'] = {"lat": latitude, "lon": longitude}
            soil_data['source'] = 'gemini_estimation'
            
            soil_cache.set(cache_key, soil_data)
            print(f"Fetched soil data from Gemini for {location_name}")
            return soil_data
        
        return {"success": False, "error": "No response from Gemini"}
        
    except json.JSONDecodeError as e:
        print(f"JSON decode error for soil: {e}")
        return {"success": False, "error": "Invalid JSON response"}
    except Exception as e:
        print(f"Gemini soil data error: {e}")
        return {"success": False, "error": str(e)}

def fetch_soil_data(latitude, longitude, location_name=""):
    """Wrapper function that uses Gemini for soil estimation"""
    return fetch_soil_data_gemini(latitude, longitude, location_name)

def get_crop_requirements_from_gemini(crop_name):
    """Dynamically fetch crop requirements using Gemini AI"""
    cache_key = f"crop_{crop_name.lower()}"
    cached = crop_cache.get(cache_key)
    if cached:
        print(f"Using cached crop data for {crop_name}")
        return cached
    
    try:
        prompt = f"""Provide detailed agricultural requirements for growing {crop_name} in JSON format.

Return ONLY valid JSON with this exact structure:
{{
    "crop_name": "{crop_name}",
    "temperature_range_celsius": {{"min": X, "max": Y}},
    "annual_rainfall_mm": {{"min": X, "max": Y}},
    "soil_ph_range": {{"min": X, "max": Y}},
    "suitable_soil_types": ["type1", "type2"],
    "growing_season": ["season1", "season2"],
    "duration_days": X,
    "water_requirement": "low/medium/high",
    "key_nutrients": ["N", "P", "K"],
    "common_regions": ["region1", "region2"]
}}

Be precise with numerical values. Use standard agricultural data."""

        response = gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.1,
                max_output_tokens=500
            )
        )
        
        if response and response.text:
            text = response.text.strip()
            text = re.sub(r'```json\s*', '', text)
            text = re.sub(r'```\s*$', '', text)
            
            crop_data = json.loads(text)
            crop_cache.set(cache_key, crop_data)
            print(f"Fetched fresh crop data for {crop_name} from Gemini")
            return crop_data
        
        return {"success": False, "error": "No response from Gemini"}
        
    except json.JSONDecodeError as e:
        print(f"JSON decode error for crop {crop_name}: {e}")
        return {"success": False, "error": "Invalid JSON response"}
    except Exception as e:
        print(f"Gemini crop data error: {e}")
        return {"success": False, "error": str(e)}

def get_batch_crop_requirements_from_gemini(crop_names):
    """Fetch requirements for multiple crops in a single Gemini API call"""
    # Check cache first
    uncached_crops = []
    cached_results = {}
    
    for crop in crop_names:
        cache_key = f"crop_{crop.lower()}"
        cached = crop_cache.get(cache_key)
        if cached:
            cached_results[crop] = cached
            print(f"Using cached crop data for {crop}")
        else:
            uncached_crops.append(crop)
    
    # If all crops are cached, return immediately
    if not uncached_crops:
        return cached_results
    
    # Fetch uncached crops in batch
    try:
        crops_list = ', '.join(uncached_crops)
        prompt = f"""Provide detailed agricultural requirements for the following crops: {crops_list}

Return ONLY valid JSON with this exact structure - an object where each key is a crop name:
{{
    "crop1": {{
        "crop_name": "crop1",
        "temperature_range_celsius": {{"min": X, "max": Y}},
        "annual_rainfall_mm": {{"min": X, "max": Y}},
        "soil_ph_range": {{"min": X, "max": Y}},
        "suitable_soil_types": ["type1", "type2"],
        "growing_season": ["season1", "season2"],
        "duration_days": X,
        "water_requirement": "low/medium/high",
        "key_nutrients": ["N", "P", "K"],
        "common_regions": ["region1", "region2"]
    }},
    "crop2": {{
        ...
    }}
}}

Provide data for ALL crops: {crops_list}. Be precise with numerical values. Use standard agricultural data."""

        response = gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.1,
                max_output_tokens=3000  # Increased for multiple crops
            )
        )
        
        if response and response.text:
            text = response.text.strip()
            text = re.sub(r'```json\s*', '', text)
            text = re.sub(r'```\s*$', '', text)
            
            batch_data = json.loads(text)
            
            # Cache individual crop data
            for crop in uncached_crops:
                crop_lower = crop.lower()
                # Try to find crop data with case-insensitive matching
                crop_data = None
                for key in batch_data.keys():
                    if key.lower() == crop_lower:
                        crop_data = batch_data[key]
                        break
                
                if crop_data:
                    cache_key = f"crop_{crop_lower}"
                    crop_cache.set(cache_key, crop_data)
                    cached_results[crop] = crop_data
                    print(f"Fetched and cached crop data for {crop}")
                else:
                    print(f"Warning: No data returned for {crop}")
            
            return cached_results
        
        print("No response from Gemini for batch crop request")
        return cached_results  # Return whatever we have cached
        
    except json.JSONDecodeError as e:
        print(f"JSON decode error for batch crops: {e}")
        # Fallback to individual requests
        print("Falling back to individual crop requests...")
        for crop in uncached_crops:
            crop_data = get_crop_requirements_from_gemini(crop)
            if 'success' not in crop_data or crop_data.get('success') != False:
                cached_results[crop] = crop_data
        return cached_results
    except Exception as e:
        print(f"Gemini batch crop data error: {e}")
        # Fallback to individual requests
        print("Falling back to individual crop requests...")
        for crop in uncached_crops:
            crop_data = get_crop_requirements_from_gemini(crop)
            if 'success' not in crop_data or crop_data.get('success') != False:
                cached_results[crop] = crop_data
        return cached_results

def analyze_crop_suitability(weather_data, soil_data, crop_requirements):
    """Analyze if crop is suitable based on weather and soil"""
    if not weather_data.get('success') or not soil_data.get('success'):
        return {"suitable": False, "reason": "Missing data"}
    
    issues = []
    score = 100
    
    # Check temperature (using 15-day data)
    forecast_days = min(15, len(weather_data['daily_forecast']))
    avg_temp = sum(d['temp_max'] + d['temp_min'] for d in weather_data['daily_forecast'][:forecast_days]) / (2 * forecast_days)
    temp_req = crop_requirements.get('temperature_range_celsius', {})
    temp_min, temp_max = temp_req.get('min', 0), temp_req.get('max', 50)
    
    if avg_temp < temp_min:
        issues.append(f"Temperature too low (avg {avg_temp:.1f}°C, needs {temp_min}-{temp_max}°C)")
        score -= 30
    elif avg_temp > temp_max:
        issues.append(f"Temperature too high (avg {avg_temp:.1f}°C, needs {temp_min}-{temp_max}°C)")
        score -= 30
    
    # Check rainfall (project from 15-day data)
    total_rain = sum(d['precipitation_mm'] for d in weather_data['daily_forecast'][:forecast_days])
    projected_annual = (total_rain / forecast_days) * 365  # Project to annual
    rain_req = crop_requirements.get('annual_rainfall_mm', {})
    rain_min, rain_max = rain_req.get('min', 0), rain_req.get('max', 5000)
    
    if projected_annual < rain_min:
        issues.append(f"Insufficient rainfall (projected {projected_annual:.0f}mm/year, needs {rain_min}-{rain_max}mm)")
        score -= 25
    elif projected_annual > rain_max:
        issues.append(f"Excess rainfall (projected {projected_annual:.0f}mm/year, needs {rain_min}-{rain_max}mm)")
        score -= 15
    
    # Check soil pH
    soil_ph = soil_data.get('ph', 7.0)
    ph_req = crop_requirements.get('soil_ph_range', {})
    ph_min, ph_max = ph_req.get('min', 5.5), ph_req.get('max', 8.0)
    
    if soil_ph < ph_min or soil_ph > ph_max:
        issues.append(f"Soil pH {soil_ph:.1f} outside optimal range ({ph_min}-{ph_max})")
        score -= 20
    
    # Check soil type
    soil_type = soil_data.get('soil_type', 'unknown')
    suitable_soils = crop_requirements.get('suitable_soil_types', [])
    if suitable_soils and soil_type not in suitable_soils:
        issues.append(f"Soil type '{soil_type}' not ideal (prefers {', '.join(suitable_soils)})")
        score -= 15
    
    return {
        "suitable": score >= 50,
        "suitability_score": max(0, score),
        "issues": issues,
        "recommendations": [] if score >= 70 else ["Consider irrigation" if projected_annual < rain_min else ""]
    }

def recommend_best_crops(location_query, session_id):
    """Main function to recommend crops based on location"""
    
    # Geocode location
    location = geocode_location(location_query)
    if not location.get('success'):
        return {
            "success": False,
            "error": f"Could not find location: {location_query}",
            "suggestions": []
        }
    
    session_manager.set_location(session_id, location)
    lat, lon = location['latitude'], location['longitude']
    
    # Fetch dynamic data (15-day weather + Gemini soil)
    weather_data = fetch_weather_data(lat, lon, days=15)
    soil_data = fetch_soil_data(lat, lon, location['name'])
    
    if not weather_data.get('success') or not soil_data.get('success'):
        return {
            "success": False,
            "error": "Could not fetch weather or soil data",
            "suggestions": []
        }
    
    # Common crops to check
    crops_to_check = ['rice', 'wheat', 'maize', 'cotton', 'sugarcane', 'soybean', 
                     'groundnut', 'tomato', 'potato', 'pulses', 'chickpea', 'mustard']
    
    # Fetch all crop requirements at once
    print(f"Fetching requirements for {len(crops_to_check)} crops in batch...")
    all_crop_requirements = get_batch_crop_requirements_from_gemini(crops_to_check)
    
    crop_analyses = []
    
    for crop in crops_to_check:
        crop_req = all_crop_requirements.get(crop)
        if not crop_req or crop_req.get('success') == False:
            print(f"Skipping {crop} - no valid data")
            continue
        
        analysis = analyze_crop_suitability(weather_data, soil_data, crop_req)
        crop_analyses.append({
            "crop": crop,
            "score": analysis['suitability_score'],
            "suitable": analysis['suitable'],
            "issues": analysis['issues'],
            "requirements": crop_req
        })
    
    # Sort by suitability score
    crop_analyses.sort(key=lambda x: x['score'], reverse=True)
    
    forecast_days = len(weather_data['daily_forecast'])
    
    return {
        "success": True,
        "location": location,
        "weather_summary": {
            "avg_temp": sum(d['temp_max'] + d['temp_min'] for d in weather_data['daily_forecast']) / (2 * forecast_days),
            "total_rainfall_15d": sum(d['precipitation_mm'] for d in weather_data['daily_forecast']),
            "avg_humidity": sum(d['humidity_percent'] for d in weather_data['daily_forecast']) / forecast_days,
            "forecast_days": forecast_days
        },
        "soil_summary": soil_data,
        "top_crops": crop_analyses[:5],
        "all_analyses": crop_analyses
    }

def fetch_crop_prices(crop_name, location_query=None):
    """Fetch current crop prices and predict future trends using Gemini AI"""
    cache_key = f"price_{crop_name.lower()}_{location_query or 'general'}"
    cached = price_cache.get(cache_key)
    if cached:
        print(f"Using cached price data for {crop_name}")
        return cached
    
    try:
        # Use Gemini to generate realistic price data and predictions
        prompt = f"""Provide current market price information and 3-month price prediction for {crop_name} in India{' ('+location_query+')' if location_query else ''}.

Return ONLY valid JSON with this exact structure:
{{
    "crop_name": "{crop_name}",
    "current_price_per_quintal": {{
        "min": X,
        "max": Y,
        "average": Z
    }},
    "currency": "INR",
    "unit": "quintal",
    "market_trend": "rising/stable/falling",
    "price_prediction_3months": {{
        "expected_min": X,
        "expected_max": Y,
        "confidence": "high/medium/low"
    }},
    "factors_affecting_price": ["factor1", "factor2"],
    "best_selling_season": "season_name",
    "data_source": "market_analysis"
}}

Use realistic Indian agricultural market prices. Base on current market conditions."""

        response = gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.1,
                max_output_tokens=400
            )
        )
        
        if response and response.text:
            text = response.text.strip()
            text = re.sub(r'```json\s*', '', text)
            text = re.sub(r'```\s*$', '', text)
            
            price_data = json.loads(text)
            price_data['success'] = True
            price_data['timestamp'] = datetime.now().isoformat()
            
            price_cache.set(cache_key, price_data)
            print(f"Fetched fresh price data for {crop_name}")
            return price_data
        
        return {"success": False, "error": "No response from Gemini"}
        
    except json.JSONDecodeError as e:
        print(f"JSON decode error for prices: {e}")
        return {"success": False, "error": "Invalid JSON response"}
    except Exception as e:
        print(f"Gemini price data error: {e}")
        return {"success": False, "error": str(e)}

def generate_contextual_followup_questions(question, response, context_data):
    """Generate 3 relevant follow-up questions using Gemini"""
    try:
        prompt = f"""Based on this agricultural conversation, suggest 3 highly relevant follow-up questions the farmer might ask next.

USER QUESTION: {question}
ASSISTANT RESPONSE: {response[:500]}...
CONTEXT: Location={context_data.get('location', 'unknown')}, Has weather data={context_data.get('has_weather', False)}

Generate 3 practical, specific questions that:
1. Build naturally on the conversation
2. Are relevant to the farmer's situation
3. Are actionable and helpful

Format as:
1. [question]
2. [question]
3. [question]

Keep each question under 15 words."""

        result = gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.7,
                max_output_tokens=200
            )
        )
        
        if result and result.text:
            questions = []
            for line in result.text.strip().split('\n'):
                line = line.strip()
                if re.match(r'^\d+\.', line):
                    question = re.sub(r'^\d+\.\s*', '', line)
                    questions.append(question)
            
            return questions[:3] if len(questions) >= 3 else questions
        
        return []
        
    except Exception as e:
        print(f"Follow-up generation error: {e}")
        return []

def detect_question_intent_with_gemini(question):
    """NEW: Use Gemini to dynamically detect if question is about crop recommendation"""
    try:
        prompt = f"""Analyze this agricultural question and determine its intent.

QUESTION: "{question}"

Classify into ONE category:
1. CROP_RECOMMENDATION - asking about best/suitable crops for a location
2. CROP_SPECIFIC - asking about a specific crop's care/diseases/cultivation
3. GENERAL_INFO - general agricultural knowledge question
4. WEATHER_QUERY - asking about weather/climate
5. SOIL_QUERY - asking about soil conditions
6. PEST_DISEASE - asking about pests or diseases
7. LOCATION_SETTING - user is setting/providing their location
8. OTHER - doesn't fit above categories

Respond with ONLY the category name (e.g., "CROP_RECOMMENDATION")."""

        response = gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.1,
                max_output_tokens=50
            )
        )
        
        if response and response.text:
            intent = response.text.strip().upper()
            print(f"[Intent Detection] Question: '{question[:50]}...' -> Intent: {intent}")
            return intent
        
        return "OTHER"
        
    except Exception as e:
        print(f"Intent detection error: {e}")
        return "OTHER"

def get_similar_questions_for_query(user_question, context_data=None):
    """Generate similar questions that users might ask - SEPARATE ENDPOINT"""
    try:
        context_str = ""
        if context_data:
            context_str = f"\nContext: Location={context_data.get('location', 'N/A')}, Topic={context_data.get('topic', 'general agriculture')}"
        
        prompt = f"""Given this agricultural question, generate 5 similar questions that farmers commonly ask on the same topic.

ORIGINAL QUESTION: "{user_question}"{context_str}

Generate 5 variations/related questions that:
1. Are on the same agricultural topic
2. Explore different angles of the same problem
3. Are practical and commonly asked by farmers
4. Vary in specificity (some general, some specific)

Format as:
1. [question]
2. [question]
3. [question]
4. [question]
5. [question]

Keep each under 20 words."""

        result = gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.6,
                max_output_tokens=300
            )
        )
        
        if result and result.text:
            similar_questions = []
            for line in result.text.strip().split('\n'):
                line = line.strip()
                if re.match(r'^\d+\.', line):
                    q = re.sub(r'^\d+\.\s*', '', line)
                    similar_questions.append(q)
            
            return {
                "success": True,
                "original_question": user_question,
                "similar_questions": similar_questions[:5],
                "count": len(similar_questions[:5])
            }
        
        return {"success": False, "error": "No response generated"}
        
    except Exception as e:
        print(f"Similar questions error: {e}")
        return {"success": False, "error": str(e)}

# ===== EXISTING FUNCTIONS (keeping essential ones) =====

def detect_language(text):
    try:
        detected_lang = detect(text)
        if detected_lang in SUPPORTED_LANGUAGES:
            return detected_lang
        return 'en'
    except:
        return 'en'

def translate_to_english(text):
    try:
        detected_lang = detect_language(text)
        if detected_lang == 'en':
            return text, detected_lang
        if detected_lang not in SUPPORTED_LANGUAGES:
            return text, 'en'
        translator = GoogleTranslator(source=detected_lang, target='en')
        translated = translator.translate(text)
        return translated, detected_lang
    except:
        return text, 'en'

def translate_from_english(text, target_lang):
    if target_lang == 'en' or target_lang == 'english':
        return text
    if target_lang not in SUPPORTED_LANGUAGES:
        return text
    try:
        translator = GoogleTranslator(source='en', target=target_lang)
        return translator.translate(text)
    except:
        return text

def clean_answer(text):
    text = re.sub(r"https?://\S+", "", text)
    text = re.sub(r"\[[^\]]*\]", "", text)
    text = re.sub(r"\([^)]*\)", "", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

def cleanup_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def get_conversation_context(session_id):
    """Get enhanced conversation context for LLM from session history"""
    conversation_history = session_manager.get_history(session_id)
    
    if not conversation_history:
        return ""
    
    context_parts = ["=== CONVERSATION HISTORY ==="]
    
    recent_exchanges = conversation_history[-3:]
    
    for i, exchange in enumerate(recent_exchanges, 1):
        timestamp = datetime.fromisoformat(exchange['timestamp']).strftime("%H:%M")
        context_parts.append(f"Exchange {i} [{timestamp}]:")
        context_parts.append(f"User: {exchange['user_input']}")
        
        if exchange.get('image_context'):
            context_parts.append(f"Image: {exchange['image_context']}")
        
        response = exchange['bot_response']
        if len(response) > 200:
            response = response[:200] + "..."
        context_parts.append(f"Assistant: {response}")
        context_parts.append("")
    
    context_parts.append("=== END HISTORY ===")
    return "\n".join(context_parts)

def analyze_user_intent_with_llm(question, conversation_context):
    """LLM-based analysis with better prompting to reduce ambiguity"""
    global your_llm_pipeline
    
    if your_llm_pipeline is None:
        return {
            'is_agricultural': True,
            'is_greeting': False,
            'references_history': False,
            'interaction_type': 'question'
        }
    
    try:
        # REFINED INTENT PROMPT: More explicit rules
        intent_prompt = f"""<s>[INST] You are an agricultural classification agent. Classify if user input is agriculture-related.

AGRICULTURAL KEYWORDS (if ANY of these appear → AGRICULTURAL: YES):
- Farm terms: crop, plant, seed, harvest, field, farm, agriculture, cultivation
- Animals: cattle, cow, buffalo, goat, sheep, pig, chicken, duck, fish, livestock, poultry
- Products: milk, dairy, egg, meat, wool, honey, silk
- Soil: fertilizer, compost, manure, soil, irrigation, water, drainage
- Problems: disease, pest, insect, weed, drought, flood, blight, rot
- Equipment: tractor, plow, harvester, sprayer, pump
- Activities: planting, sowing, grazing, milking, breeding, aquaculture
- Crops: rice, wheat, corn, tomato, potato, sugarcane, cotton, tea, coffee

STRICT CLASSIFICATION RULES:
1. If question contains ANY agricultural keyword → AGRICULTURAL: YES
2. Questions about farming methods/techniques → AGRICULTURAL: YES
3. Questions about growing/raising food → AGRICULTURAL: YES
4. Simple definitions like "what is crop rotation" → AGRICULTURAL: YES
5. ONLY mark NON-agricultural if clearly about: sports, movies, politics, technology, general knowledge with NO farming context

GREETING DETECTION (mark YES only if):
- Pure social: "hi", "hello", "how are you", "thanks", "bye"
- Questions with agricultural words are NOT greetings

HISTORY REFERENCE (mark YES only if):
- Uses pronouns: "it", "this", "that", "the same"
- Phrases: "tell me more", "what about this", "continue"

CONVERSATION CONTEXT:
{conversation_context}

USER INPUT: "{question}"

Respond in this EXACT format (no extra text):
AGRICULTURAL: [YES/NO]
GREETING: [YES/NO]
REFERENCES_HISTORY: [YES/NO]
INTERACTION_TYPE: [QUESTION/GREETING/THANKS/FOLLOWUP]
REASONING: [One sentence explaining your decision]
[/INST]"""

        result = your_llm_pipeline(
            intent_prompt,
            max_new_tokens=80,
            do_sample=False,
            temperature=0.05,
            pad_token_id=your_llm_pipeline.tokenizer.eos_token_id
        )
        
        if result and len(result) > 0:
            response = result[0]['generated_text'].strip()
            
            intent_analysis = {
                'is_agricultural': 'AGRICULTURAL: YES' in response.upper(),
                'is_greeting': 'GREETING: YES' in response.upper(),
                'references_history': 'REFERENCES_HISTORY: YES' in response.upper(),
                'interaction_type': 'question'
            }
            
            for line in response.split('\n'):
                if line.startswith('INTERACTION_TYPE:'):
                    intent_analysis['interaction_type'] = line.split(':', 1)[1].strip().lower()
                    break
            
            print(f"LLM Intent Analysis: {intent_analysis}")
            return intent_analysis
        
    except Exception as e:
        print(f"Intent analysis error: {e}")
    
    return {
        'is_agricultural': True,
        'is_greeting': False,
        'references_history': False,
        'interaction_type': 'question'
    }

def generate_contextual_response_with_llm(intent_analysis, question, conversation_context, original_lang='en'):
    """Better greeting/redirect responses"""
    global your_llm_pipeline
    
    if your_llm_pipeline is None:
        return "Hello! I'm your agricultural assistant. How can I help you today?"
    
    try:
        if intent_analysis['is_greeting'] or intent_analysis['interaction_type'] in ['greeting', 'thanks', 'goodbye']:
            response_prompt = f"""<s>[INST] You are a friendly agricultural assistant. User sent a greeting/social message.

USER MESSAGE: "{question}"
TYPE: {intent_analysis['interaction_type']}

Generate a SHORT, warm response (2-3 sentences) that:
1. Responds naturally to their message
2. Mentions you help with farming/agriculture  
3. Invites a question

Keep it brief and friendly. Don't list all topics.

Response: [/INST]"""
        else:
            # Non-agricultural query redirect
            response_prompt = f"""<s>[INST] You are an agricultural assistant. User asked a non-agricultural question.

USER QUESTION: "{question}"

Generate a polite response (2-3 sentences) that:
1. Politely says you specialize in agriculture/farming
2. Briefly mention what you can help with (crops, livestock, soil, etc.)
3. Invite them to ask farming questions

Keep it short and encouraging.

Response: [/INST]"""
        
        result = your_llm_pipeline(
            response_prompt,
            max_new_tokens=150,
            temperature=0.2,
            do_sample=False,
            top_p=0.9,
            pad_token_id=your_llm_pipeline.tokenizer.eos_token_id
        )
        
        if result and len(result) > 0:
            response = result[0]['generated_text'].strip()
            
            if original_lang != 'en':
                response = translate_from_english(response, original_lang)
            
            return response
        
    except Exception as e:
        print(f"Response generation error: {e}")
    
    fallback = "Hello! I'm your agricultural assistant for crops, livestock, soil, and farming. What would you like to know?"
    if original_lang != 'en':
        fallback = translate_from_english(fallback, original_lang)
    return fallback

def rewrite_query_with_history(follow_up_question, history):
    """LLM-based query rewriting for conversational context"""
    global your_llm_pipeline
    
    if not history or your_llm_pipeline is None:
        return follow_up_question
        
    history_context = []
    for exchange in history[-2:]:
        user_input_en, _ = translate_to_english(exchange['user_input'])
        bot_response_en, _ = translate_to_english(exchange['bot_response'])
        
        bot_response_summary = (bot_response_en[:200] + '...') if len(bot_response_en) > 200 else bot_response_en
        
        history_context.append(f"User: {user_input_en}")
        history_context.append(f"Assistant: {bot_response_summary}")
        
    history_str = "\n".join(history_context)

    rewrite_prompt = f"""<s>[INST] Rewrite follow-up question into standalone question using conversation history.

HISTORY:
{history_str}

FOLLOW-UP: "{follow_up_question}"

Rewrite as clear standalone question. Example: "tell me more about it" → "Tell me more about tomato blight"

STANDALONE QUESTION: [/INST]"""

    try:
        result = your_llm_pipeline(
            rewrite_prompt,
            max_new_tokens=60,
            do_sample=False,
            temperature=0.0,
            pad_token_id=your_llm_pipeline.tokenizer.eos_token_id
        )
        
        if result and len(result) > 0:
            rewritten_query = result[0]['generated_text'].strip().replace('"', '')
            
            if len(rewritten_query) > len(follow_up_question) + 3 and len(rewritten_query) < 200:
                print(f"Rewritten query: '{rewritten_query}'")
                return rewritten_query
        
        return follow_up_question
            
    except Exception as e:
        print(f"Error during query rewriting: {e}")
        return follow_up_question

def generate_detailed_analysis(initial_answer, question, gemini_analysis="", conversation_context="", original_lang='en'):
    """ENHANCED detailed analysis - EXACTLY 400 WORDS with 800 tokens"""
    global your_llm_pipeline
    
    if your_llm_pipeline is None:
        return initial_answer + "\n\n[Note: Detailed analysis unavailable]"
    
    try:
        # UPDATED PROMPT: Explicitly request 400 words
        detailed_prompt = f"""<s>[INST] You are a senior agricultural consultant with 20+ years of field experience. Provide ONE comprehensive expert analysis in EXACTLY 400 WORDS.

QUESTION: {question}
BASIC ANSWER: {initial_answer}
IMAGE ANALYSIS: {gemini_analysis}
CONTEXT: {conversation_context}

CRITICAL INSTRUCTIONS:
1. Write EXACTLY 400 words - not more, not less
2. Write ONE unified expert response - NOT separate sections or bullet points
3. Focus ONLY on what's most relevant to the specific question
4. Keep it practical and actionable - avoid theoretical tangents
5. Flow naturally from problem identification → solution → prevention
6. Be specific with recommendations (exact products, dosages, timing)
7. Write in cohesive paragraph style

STRUCTURE YOUR RESPONSE AS A FLOWING NARRATIVE:
- Start with the core issue/topic and its immediate significance
- Explain the underlying cause or mechanism briefly
- Provide specific, actionable solution steps with exact details
- Include timing, dosage, or frequency where applicable
- Mention 2-3 preventive measures naturally in the flow
- End with one practical monitoring/follow-up tip

AVOID:
- Don't create separate numbered sections or headers
- Don't jump between multiple unrelated aspects
- Don't give generic advice - be specific to THIS question
- Don't use bullet points - write in paragraph form
- Don't repeat information from the basic answer unnecessarily

Write a SINGLE cohesive expert consultation response in EXACTLY 400 WORDS:

[/INST]"""

        detailed_result = your_llm_pipeline(
            detailed_prompt,
            max_new_tokens=800,  # UPDATED: 800 tokens for ~400 words
            temperature=0.2,
            do_sample=False,
            top_p=0.85,
            return_full_text=False
        )
        
        if detailed_result and len(detailed_result) > 0:
            detailed_analysis = detailed_result[0]['generated_text'].strip()
            
            # Clean up any remaining headers or bullets that might appear
            detailed_analysis = re.sub(r'^\d+\.\s+\*\*.*?\*\*:?\s*', '', detailed_analysis, flags=re.MULTILINE)
            detailed_analysis = re.sub(r'^\*\*.*?\*\*:?\s*', '', detailed_analysis, flags=re.MULTILINE)
            detailed_analysis = re.sub(r'^[-•]\s+', '', detailed_analysis, flags=re.MULTILINE)
            
            if len(detailed_analysis) > 200:
                enhanced_response = detailed_analysis
                
                if original_lang != 'en':
                    try:
                        enhanced_response = translate_from_english(enhanced_response, original_lang)
                    except Exception:
                        enhanced_response += f"\n\n[Note: Analysis provided in English due to translation limitations]"
                
                return enhanced_response
        
        return initial_answer 
            
    except Exception as e:
        print(f"Error in detailed analysis: {e}")
        return initial_answer + f"\n\n**Note:** Enhanced analysis encountered an error."

def initialize_speech_recognition():
    global recognizer
    try:
        recognizer = sr.Recognizer()
        recognizer.energy_threshold = 4000
        recognizer.dynamic_energy_threshold = True
        return True
    except:
        return False

def initialize_gemini_model():
    global gemini_model
    try:
        genai.configure(api_key=GEMINI_API_KEY)
        test_model = genai.GenerativeModel('gemini-2.0-flash')
        test_response = test_model.generate_content("Test")
        if test_response and test_response.text:
            gemini_model = test_model
            return True
        return False
    except Exception as e:
        print(f"Gemini init error: {e}")
        return False

def analyze_agricultural_image_with_gemini(image):
    global gemini_model
    if image is None or gemini_model is None:
        return "", "", False
    
    try:
        if not isinstance(image, Image.Image):
            image = Image.fromarray(image).convert('RGB')
        else:
            image = image.convert('RGB')
        
        max_size = (1024, 1024)
        if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
            image.thumbnail(max_size, Image.Resampling.LANCZOS)
        
        prompt = """Analyze this agricultural image precisely.

AGRICULTURAL: [YES/NO]
CATEGORY: [CROP/SOIL/LIVESTOCK/PEST/EQUIPMENT/DISEASE]
IDENTIFICATION: [Specific name]
CONDITION: [Specific problem or "Healthy"]
CONFIDENCE: [HIGH/MEDIUM/LOW]

If NOT agricultural, respond:
AGRICULTURAL: NO
REASON: [Why not]"""

        response = gemini_model.generate_content(
            [prompt, image],
            generation_config=genai.types.GenerationConfig(temperature=0.1, max_output_tokens=800)
        )
        
        if not response or not response.text:
            return "Analysis failed", "Unknown", False
        
        analysis_text = response.text.strip()
        is_agricultural = 'AGRICULTURAL: YES' in analysis_text.upper()
        
        identification = ""
        for line in analysis_text.split('\n'):
            if line.startswith('IDENTIFICATION:'):
                identification = line.split(':', 1)[1].strip() if ':' in line else ""
        
        return analysis_text, identification or "Agricultural subject", is_agricultural
            
    except Exception as e:
        return f"Error: {str(e)}", "Error", False

def load_vectorstore(index_path):
    try:
        embedder = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': 'cuda:0' if torch.cuda.is_available() else 'cpu'}
        )
        vectorstore = FAISS.load_local(index_path, embedder, allow_dangerous_deserialization=True)
        return vectorstore
    except Exception as e:
        print(f"Vectorstore load error: {e}")
        return None

def build_qa_chain(vectorstore):
    global your_llm_pipeline
    try:
        model_id = "mistralai/Mistral-7B-Instruct-v0.2"
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        tokenizer.pad_token = tokenizer.eos_token
        
        device_map = "cuda:0" if torch.cuda.is_available() else "cpu"
        torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
        
        model = AutoModelForCausalLM.from_pretrained(
            model_id, torch_dtype=torch_dtype, device_map=device_map,
            trust_remote_code=True, low_cpu_mem_usage=True, use_cache=True
        ).eval()
        
        pipe = pipeline(
            "text-generation", model=model, tokenizer=tokenizer,
            max_new_tokens=256, temperature=0.1, do_sample=False,
            return_full_text=False, pad_token_id=tokenizer.eos_token_id, batch_size=1
        )
        
        your_llm_pipeline = pipe
        llm = HuggingFacePipeline(pipeline=pipe)
        retriever = vectorstore.as_retriever(search_type="similarity", k=2)
        return RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    except Exception as e:
        print(f"QA chain error: {e}")
        return None

# ===== MAIN ANSWER FUNCTION WITH DYNAMIC DATA & ENHANCED LOCATION =====

def answer_question_core(question, image=None, use_detailed_analysis=False, actual_session_id=None):
    global qa_chain
    
    if qa_chain is None:
        return "System not initialized"
    
    if not question.strip() and image is None:
        return "Please enter a question or upload an image"
    
    try:
        original_question = question
        original_lang = 'en'
        
        # Language detection and translation
        if question.strip():
            original_lang = detect_language(question.strip())
            if original_lang != 'en':
                english_question, _ = translate_to_english(question.strip())
                question = english_question
        
        # Get conversation context for intent analysis
        conversation_context = get_conversation_context(actual_session_id)
        
        # ===== STEP 1: ANALYZE USER INTENT WITH LLM (from version1) =====
        intent_analysis = analyze_user_intent_with_llm(question, conversation_context)
        
        # Handle greetings
        if intent_analysis['is_greeting'] or intent_analysis['interaction_type'] in ['greeting', 'thanks', 'goodbye']:
            response = generate_contextual_response_with_llm(intent_analysis, question, conversation_context, original_lang)
            session_manager.add_to_history(actual_session_id, original_question, response, "", original_lang, use_detailed_analysis, intent_analysis['interaction_type'])
            return response
        
        # ===== STEP 2: DETECT QUESTION INTENT DYNAMICALLY =====
        question_intent = detect_question_intent_with_gemini(question)
        
        # ===== STEP 2: HANDLE LOCATION SETTING =====
        if question_intent == "LOCATION_SETTING":
            location_setting_patterns = [
                r'i am (?:in|at|from|near)\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)',
                r'my (?:location|place|city|area) is\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)',
                r'(?:set|save|remember) (?:my )?location (?:as|to)?\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)'
            ]
            
            for pattern in location_setting_patterns:
                match = re.search(pattern, question, re.IGNORECASE)
                if match:
                    new_location = match.group(1)
                    # Geocode and save location
                    geocoded = geocode_location(new_location)
                    if geocoded.get('success'):
                        session_manager.set_location(actual_session_id, geocoded)
                        response = f"""✅ **Location Saved Successfully!**

📍 **Location:** {geocoded['name']}
🌐 **Coordinates:** {geocoded['latitude']:.4f}°N, {geocoded['longitude']:.4f}°E

Now you can ask questions using "here", "my location", or "my area"!

**Try these:**
- "What is the best crop here?"
- "Show me weather forecast for my area"
- "Which crops suit my location?"
- "What's the soil like here?"

Your location is saved for this session only."""
                        
                        if original_lang != 'en':
                            response = translate_from_english(response, original_lang)
                        
                        session_manager.add_to_history(
                            actual_session_id, original_question, response, "", 
                            original_lang, use_detailed_analysis, "location_setting",
                            new_location, None
                        )
                        
                        return response
                    else:
                        return f"❌ Could not find location: {new_location}. Please check spelling or try a nearby city."
        
        # ===== STEP 3: DETECT LOCATION IN QUESTION =====
        location_query = None
        
        # First try explicit location patterns (case-insensitive, limit to 1-3 words)
        location_patterns = [
            r'in\s+([A-Za-z]+(?:\s+[A-Za-z]+){0,2})(?:\s|,|\?|$)',
            r'at\s+([A-Za-z]+(?:\s+[A-Za-z]+){0,2})(?:\s|,|\?|$)',
            r'for\s+([A-Za-z]+(?:\s+[A-Za-z]+){0,2})(?:\s|,|\?|$)',
            r'near\s+([A-Za-z]+(?:\s+[A-Za-z]+){0,2})(?:\s|,|\?|$)',
            r'around\s+([A-Za-z]+(?:\s+[A-Za-z]+){0,2})(?:\s|,|\?|$)'
        ]
        
        for pattern in location_patterns:
            match = re.search(pattern, question, re.IGNORECASE)
            if match:
                potential_location = match.group(1).strip()
                # Filter out common non-location words
                non_location_words = ['here', 'there', 'my', 'this', 'that', 'the', 'a', 'an', 
                                     'which', 'what', 'where', 'when', 'how', 'best', 'good',
                                     'suitable', 'crops', 'crop', 'farming', 'agriculture',
                                     'growing', 'cultivation', 'season', 'weather', 'soil']
                if potential_location.lower() not in non_location_words:
                    location_query = potential_location
                    print(f"[Location] Explicit location found: {location_query}")
                    break
        
        # ===== STEP 4: HANDLE CONTEXT-DEPENDENT LOCATION WORDS =====
        if not location_query:
            context_location_words = ['here', 'my location', 'my area', 'nearby', 
                                     'this place', 'current location', 'my region',
                                     'this area', 'this region', 'local']
            
            question_lower = question.lower()
            if any(word in question_lower for word in context_location_words):
                # Try to retrieve location from session cache
                cached_location = session_manager.get_location(actual_session_id)
                if cached_location and isinstance(cached_location, dict):
                    location_query = cached_location.get('name', None)
                    print(f"[Session {actual_session_id[-8:]}] Using cached location: {location_query}")
                else:
                    # No cached location - ask user to specify
                    prompt_response = """🔍 **Location Required**

I understand you're asking about crops for your location ("here"), but I need to know your specific location to provide accurate recommendations with **real-time weather and soil data**.

**Please do ONE of the following:**

**Option 1:** Specify location in your question
- Example: "What is the best crop in Pune?"
- Example: "Best crops for Mumbai, Maharashtra"

**Option 2:** Set your location first
- Say: "I am in [Your City/District]"
- Example: "I am in Bangalore"

**Option 3:** Ask a general question
- I'll provide general guidelines without location-specific data

---

💡 **Why location matters:**
✅ Real-time 15-day weather forecast
✅ AI-powered soil analysis for your area
✅ Crop suitability scores (0-100)
✅ Location-specific recommendations

Once you set your location, you can use "here" in all future questions!"""
                    
                    if original_lang != 'en':
                        prompt_response = translate_from_english(prompt_response, original_lang)
                    
                    return prompt_response
        
        # ===== STEP 5: DYNAMIC CROP RECOMMENDATION PATH =====
        if question_intent == "CROP_RECOMMENDATION" and location_query:
            print(f"[Dynamic] Fetching live data for: {location_query}")
            crop_recommendation = recommend_best_crops(location_query, actual_session_id)
            
            if crop_recommendation.get('success'):
                dynamic_data = crop_recommendation
                
                # Format response with live data
                weather = crop_recommendation['weather_summary']
                soil = crop_recommendation['soil_summary']
                top_crops = crop_recommendation['top_crops'][:3]
                forecast_days = weather.get('forecast_days', 15)
              
                # Brief Weather Summary
                response = f"""**Weather (Next {forecast_days} Days):** {weather['avg_temp']:.1f}°C avg, {weather['total_rainfall_15d']:.1f}mm rain, {weather['avg_humidity']:.0f}% humidity\n\n"""
                
                # Brief Soil Summary
                response += f"""**Soil:** {soil['soil_type'].replace('_', ' ').title()}, pH {soil['ph']:.1f}, {soil.get('fertility', 'Medium').title()} fertility\n\n"""
                
                response += f"""**TOP 3 RECOMMENDED CROPS:**\n\n"""
                
                for i, crop_data in enumerate(top_crops, 1):
                    crop_name = crop_data['crop'].title()
                    score = crop_data['score']
                    suitable = "HIGHLY SUITABLE" if score >= 80 else "MODERATELY SUITABLE" if score >= 60 else "NOT RECOMMENDED"
                    
                    response += f"""**{i}. {crop_name}** - Score: {score}/100 {suitable}\n"""
                    
                    req = crop_data['requirements']
                    response += f"""   • Duration: {req.get('duration_days', 'N/A')} days | Water: {req.get('water_requirement', 'N/A').title()}\n"""
                    
                    # Fetch and add price prediction
                    price_data = fetch_crop_prices(crop_data['crop'], location_query)
                    if price_data.get('success'):
                        current_avg = price_data['current_price_per_quintal']['average']
                        trend = price_data['market_trend']
                        pred_avg = (price_data['price_prediction_3months']['expected_min'] + price_data['price_prediction_3months']['expected_max']) / 2
                        trend_icon = "📈" if trend == "rising" else "📉" if trend == "falling" else "➡️"
                        response += f"""   • Current Price: ₹{current_avg:.0f}/quintal {trend_icon} | 3-Month Forecast: ₹{pred_avg:.0f}/quintal\n"""
                    
                    if crop_data['issues']:
                        response += f""" Note: {crop_data['issues'][0]}\n"""
                    
                    response += "\n"
                
                # Generate follow-up questions
                followup_questions = generate_contextual_followup_questions(
                    question, 
                    response,
                    {
                        'location': location_query,
                        'has_weather': True,
                        'top_crop': top_crops[0]['crop'] if top_crops else None
                    }
                )
                
                if followup_questions:
                    response += "\n\n**💡Similar questions:**\n"
                    for i, fq in enumerate(followup_questions, 1):
                        response += f"{i}. {fq}\n"
                
                # Translate if needed
                if original_lang != 'en':
                    try:
                        response = translate_from_english(response, original_lang)
                    except:
                        response += "\n\n[Note: Response in English due to translation limits]"
                
                # Save to history
                session_manager.add_to_history(
                    actual_session_id, original_question, response, "", 
                    original_lang, use_detailed_analysis, "crop_recommendation",
                    location_query, dynamic_data
                )
                
                return response
            else:
                # Failed to get crop recommendation data
                error_msg = crop_recommendation.get('error', 'Unknown error')
                return f"❌ Unable to fetch live data: {error_msg}\n\nPlease try again or ask a general question."
        
        # ===== STEP 6: IMAGE ANALYSIS =====
        gemini_full_analysis = ""
        gemini_summary = ""
        image_is_agricultural = False
        
        if image is not None:
            print("Analyzing image with Gemini...")
            gemini_full_analysis, gemini_summary, image_is_agricultural = analyze_agricultural_image_with_gemini(image)
            
            if not image_is_agricultural:
                response = f"""Non-Agricultural Image Detected

The uploaded image doesn't appear to be related to agriculture, farming, livestock, dairy, fisheries, horticulture, or other agricultural topics.

Analysis: {gemini_full_analysis}

I specialize in agricultural topics including:
- Crops and plant cultivation
- Livestock and animal husbandry  
- Dairy farming and milk production
- Fisheries and aquaculture
- Horticulture and gardening
- Forestry and tree cultivation
- Soil management and fertilization
- Agricultural equipment and machinery
- Plant and animal diseases
- Pest management

Please upload an agriculture-related image!"""
                
                if original_lang != 'en':
                    response = translate_from_english(response, original_lang)
                
                session_manager.add_to_history(
                    actual_session_id, original_question, response, 
                    "Non-agricultural image", original_lang, use_detailed_analysis
                )
                return response
        
        # ===== REFINED: Better non-agricultural detection (from version1) =====
        if not intent_analysis['is_agricultural'] and not image_is_agricultural:
            response = generate_contextual_response_with_llm(intent_analysis, question, conversation_context, original_lang)
            session_manager.add_to_history(actual_session_id, original_question, response, "", original_lang, use_detailed_analysis)
            return response
        
        # ===== QUERY REWRITING WITH HISTORY (from version1) =====
        conversation_history = session_manager.get_history(actual_session_id)
        if intent_analysis['references_history']:
            rewritten_question = rewrite_query_with_history(question, conversation_history)
            if rewritten_question.lower() != question.lower():
                print(f"[History Rewrite] Original: '{question}' → Rewritten: '{rewritten_question}'")
                question = rewritten_question
        
        # ===== STEP 7: STANDARD RAG QUERY WITH CONVERSATION CONTEXT =====
        rag_query_parts = []
        
        # Add current question
        if question.strip():
            rag_query_parts.append(f"Current question: {question.strip()}")
        elif image is not None:
            rag_query_parts.append(f"Analyzing uploaded image showing: {gemini_summary}")
        
        # Add image analysis if available
        if gemini_summary:
            rag_query_parts.append(f"Image analysis shows: {gemini_summary}")
        
        # Add conversation context for better RAG retrieval
        if conversation_context:
            rag_query_parts.append(conversation_context)
        
        comprehensive_query = "\n\n".join(rag_query_parts)
        
        print(f"[Session {actual_session_id[-8:]}] Querying RAG with intent: {question_intent}")
        rag_result = qa_chain.run(comprehensive_query)
        rag_result = clean_answer(rag_result)
        
        final_response = rag_result
        
        # ===== APPLY EXPERT ANALYSIS IF REQUESTED (from version1) =====
        if use_detailed_analysis:
            print(f"[Session {actual_session_id[-8:]}] Applying detailed analysis...")
            final_response = generate_detailed_analysis(
                rag_result, 
                question, 
                gemini_summary, 
                conversation_context, 
                'en'
            )
        
        cleanup_memory()
        
        # Add image analysis if available (from version1 format)
        if gemini_full_analysis and image_is_agricultural:
            lines = gemini_full_analysis.split('\n')
            category = ""
            identification = ""
            condition = ""
            
            for line in lines:
                if line.startswith('CATEGORY:'):
                    category = line.split(':', 1)[1].strip()
                elif line.startswith('IDENTIFICATION:'):
                    identification = line.split(':', 1)[1].strip()
                elif line.startswith('CONDITION:'):
                    condition = line.split(':', 1)[1].strip()
            
            detection_header = f"**{category.title()} Detected:**" if use_detailed_analysis else f"**{category.title()}:**"
            
            response = f"{detection_header} {identification}"
            if condition and condition.lower() not in ["healthy", "normal", "none"]:
                response += f" - {condition}"
            
            response += f"\n\n{final_response}"
            final_response = response
        
        # Generate contextual follow-ups
        followup_questions = generate_contextual_followup_questions(
            question, 
            final_response,
            {
                'location': session_manager.get_location(actual_session_id),
                'has_weather': False,
                'has_image': image is not None
            }
        )
        
        if followup_questions:
            final_response += "\n\n**💡Similar questions:**\n"
            for i, fq in enumerate(followup_questions, 1):
                final_response += f"{i}. {fq}\n"
        
        cleanup_memory()
        
        if original_lang != 'en':
            try:
                final_response = translate_from_english(final_response, original_lang)
            except:
                final_response += "\n\n[Note: Response in English]"
        
        # Determine question type for history tracking
        question_type = intent_analysis.get('interaction_type', 'question')
        if intent_analysis['references_history']:
            question_type = 'followup'
        
        session_manager.add_to_history(
            actual_session_id, original_question, final_response, 
            gemini_summary, original_lang, use_detailed_analysis, 
            question_type, location_query, None
        )
        
        return final_response
            
    except Exception as e:
        cleanup_memory()
        return f"❌ Error: {str(e)}"

def answer_question(question, image=None, use_detailed_analysis=False, provided_session_id=None):
    """CONCURRENT-SAFE answer function"""
    final_session_id = provided_session_id or "default_session"
    actual_session_id = session_manager.get_session_id(request=None, provided_session_id=final_session_id)
    
    print(f"[Session {actual_session_id[-8:]}] Request received")
    session_manager.increment_active_requests(actual_session_id)
    
    try:
        with request_semaphore:
            print(f"[Session {actual_session_id[-8:]}] Processing...")
            result = answer_question_core(question, image, use_detailed_analysis, actual_session_id)
            print(f"[Session {actual_session_id[-8:]}] Completed")
            return result
    finally:
        session_manager.decrement_active_requests(actual_session_id)

# ===== API FUNCTIONS =====

def fetch_session_id_api(request: gr.Request = None):
    return session_manager.fetch_session_id(request)

def answer_question_with_session(question, image=None, use_detailed_analysis=False, session_id=None):
    return answer_question(question, image, use_detailed_analysis, session_id)

def get_conversation_history_by_session(session_id):
    if not session_id:
        return {"status": "error", "message": "Session ID required"}
    try:
        actual_session_id = session_manager.get_session_id(request=None, provided_session_id=session_id)
        history = get_conversation_history_display(actual_session_id)
        return {"status": "success", "history": history, "session_id": session_id}
    except Exception as e:
        return {"status": "error", "message": str(e)}

def clear_conversation_history_by_session(session_id):
    if not session_id:
        return {"status": "error", "message": "Session ID required"}
    try:
        actual_session_id = session_manager.get_session_id(request=None, provided_session_id=session_id)
        session_manager.clear_session_history(actual_session_id)
        return {"status": "success", "message": f"History cleared for session: {session_id}"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

def get_similar_questions_api(user_question, location=None, topic=None):
    """API endpoint for getting similar questions - SEPARATE ENDPOINT"""
    try:
        context_data = {}
        if location:
            context_data['location'] = location
        if topic:
            context_data['topic'] = topic
        
        result = get_similar_questions_for_query(user_question, context_data if context_data else None)
        return result
    except Exception as e:
        return {"success": False, "error": str(e)}

def initialize_system():
    global qa_chain
    try:
        cleanup_memory()
        
        print("Initializing speech recognition...")
        initialize_speech_recognition()
        
        print("Initializing Gemini model...")
        gemini_success = initialize_gemini_model()
        if not gemini_success:
            return "Error: Gemini API initialization failed"
        
        if not os.path.exists(INDEX_PATH):
            return "Error: FAISS index not found"
            
        print("Loading FAISS index...")
        vectordb = load_vectorstore(INDEX_PATH)
        if vectordb is None:
            return "Error: Failed to load FAISS index"
        
        print("Loading Mistral model...")
        qa_chain = build_qa_chain(vectordb)
        if qa_chain is None:
            return "Error: Failed to build QA chain"
        
        cleanup_memory()
        
        supported_langs = ", ".join([f"{name}" for name in SUPPORTED_LANGUAGES.values()])
        
        return f"""✅ ENHANCED REAL-TIME DYNAMIC AgroRAG System Ready!

🚀 **ALL FEATURES FROM VERSION1 + VERSION2:**
✓ LLM-driven agricultural intelligence (version1)
✓ Expert analysis mode (EXACTLY 400 words, 800 tokens) (version1)
✓ Non-agricultural question restriction (version1)
✓ Query rewriting with conversation history (version1)
✓ Stricter agricultural image classification (version1)
✓ Intent analysis with LLM (version1)
✓ Greeting/thanks handling (version1)
✓ Dynamic Intent Detection (Gemini AI) (version2)
✓ "Here" location support with session memory (version2)
✓ Location setting: "I am in [City]" (version2)
✓ Concurrent processing ({MAX_CONCURRENT_REQUESTS} parallel)
✓ Live weather data - 15 days (Open-Meteo API) (version2)
✓ AI-powered soil analysis (Gemini AI) (version2)
✓ Dynamic crop requirements (Gemini AI) (version2)
✓ Crop price prediction with 3-month forecast (Gemini AI) (version2)
✓ Contextual follow-up suggestions (version2)
✓ Similar questions API endpoint (version2)
✓ Location-based recommendations (version2)
✓ Multilingual: {supported_langs}

**Try asking:**
- "I am in Pune" (sets location)
- "What is the best crop here?" (uses saved location)
- "Best crops for Mumbai" (gets weather, soil, prices)
- "Tell me more about it" (rewrites with history)
- "Hello" (gets friendly greeting)

**API Endpoints:**
- get_similar_questions_api(question, location, topic)
- fetch_crop_prices(crop_name, location)
"""
            
    except Exception as e:
        return f"Error initializing system: {str(e)}"

print("Initializing ENHANCED REAL-TIME DYNAMIC AgroRAG System...")
init_status = initialize_system()
print(init_status)

# ===== UTILITY FUNCTIONS =====

def transcribe_audio(audio):
    if audio is None or recognizer is None:
        return "Audio unavailable"
    try:
        sample_rate, audio_data = audio
        if audio_data.dtype == np.int16:
            audio_data = audio_data.astype(np.float32) / 32768.0
        audio_data = (audio_data * 32767).astype(np.int16)
        
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp_file:
            with wave.open(tmp_file.name, 'wb') as wav_file:
                wav_file.setnchannels(1 if len(audio_data.shape) == 1 else audio_data.shape[1])
                wav_file.setsampwidth(2)
                wav_file.setframerate(sample_rate)
                wav_file.writeframes(audio_data.tobytes())
            
            with sr.AudioFile(tmp_file.name) as source:
                recognizer.adjust_for_ambient_noise(source, duration=0.5)
                audio = recognizer.record(source)
                try:
                    return recognizer.recognize_google(audio)
                except:
                    return "Could not understand audio"
        
        try:
            os.unlink(tmp_file.name)
        except:
            pass
    except Exception as e:
        return f"Error: {str(e)}"

def transcribe_only(audio):
    if audio is not None:
        return transcribe_audio(audio)
    return ""

def get_combined_input(transcription, text_input_val):
    if transcription and transcription.strip():
        return transcription.strip()
    elif text_input_val and text_input_val.strip():
        return text_input_val.strip()
    return ""

def process_basic_question(transcription, text_input_val, image, session_id, request: gr.Request = None):
    if session_id and session_id.strip():
        final_session_id = session_id.strip()
    else:
        final_session_id = session_manager.get_session_id(request)
    
    question = get_combined_input(transcription, text_input_val)
    if not question and image is None:
        return "Please provide a question or upload an image."
    
    return answer_question(question, image, use_detailed_analysis=False, provided_session_id=final_session_id)

def process_expert_question(transcription, text_input_val, image, session_id, request: gr.Request = None):
    """Process question with expert analysis (Web UI) - CONCURRENT SAFE"""
    if session_id and session_id.strip():
        final_session_id = session_id.strip()
    else:
        final_session_id = session_manager.get_session_id(request)
    
    question = get_combined_input(transcription, text_input_val)
    if not question and image is None:
        return "Please provide a question or upload an image."
    
    result = answer_question(question, image, use_detailed_analysis=True, provided_session_id=final_session_id)
    return f"**EXPERT ANALYSIS (400 words)** \n\n{result}"

def clear_all_inputs():
    return "", "", None, ""

def refresh_history(request: gr.Request):
    session_id = session_manager.get_session_id(request)
    return get_conversation_history_display(session_id)

def clear_conversation_history(request: gr.Request):
    session_id = session_manager.get_session_id(request)
    session_manager.clear_session_history(session_id)
    return f"✅ Your conversation history has been cleared!\n\nSession ID: {session_id}"

def update_system_status(request: gr.Request = None):
    stats = session_manager.get_session_stats()
    current_session_id = session_manager.get_session_id(request) if request else "Unknown"
    
    return f"""✅ ENHANCED REAL-TIME DYNAMIC AgroRAG System: Active

🚀 **Features:**
- Dynamic Intent Detection: Gemini AI ✓
- "Here" Location Support: Session Memory ✓
- Location Setting: "I am in [City]" ✓
- Max Concurrent Requests: {MAX_CONCURRENT_REQUESTS}
- Live Weather API: Open-Meteo (15 days) ✓
- AI Soil Estimation: Gemini AI ✓
- Dynamic Crop Data: Gemini AI ✓
- Contextual Follow-ups: Enabled ✓

📊 **System Stats:**
- Active Sessions: {stats['total_sessions']}
- Currently Processing: {stats['active_requests']} requests
- Total Exchanges: {stats['total_exchanges']}
- Your Session: {current_session_id[-8:]}

🕐 Last Updated: {datetime.now().strftime('%H:%M:%S')}"""

def get_conversation_history_display(session_id):
    """Get formatted conversation history for display for a specific session"""
    conversation_history = session_manager.get_history(session_id)
    if not conversation_history:
        return f"No conversation history yet. Start by asking an agriculture-related question!\n\n"
    
    history_text = f"## 📜 Session History (ID: {session_id[-8:]}):\n\n"
    
    for i, exchange in enumerate(conversation_history[-5:], 1):
        timestamp = datetime.fromisoformat(exchange['timestamp']).strftime("%H:%M:%S")
        lang_name = SUPPORTED_LANGUAGES.get(exchange['language'], 'Unknown')
        q_type = exchange.get('question_type', 'unknown').replace('_', ' ').title()
        
        # Add special indicators
        lang_flag = f" [{lang_name}]" if exchange['language'] != 'en' else ""
        detailed_flag = " [Expert 400w]" if exchange.get('is_detailed') else ""
        followup_flag = " 🔄" if q_type.lower() == 'followup' else ""
        
        history_text += f"**{i}. [{timestamp}]{lang_flag}{detailed_flag}{followup_flag} [{q_type}]**\n"
        history_text += f"User: {exchange['user_input']}\n"
        
        if exchange.get('location'):
            history_text += f"📍 Location: {exchange['location']}\n"
        
        if exchange.get('image_context'):
            history_text += f"🖼️ Image: {exchange['image_context']}\n"
        
        response = exchange['bot_response']
        if len(response) > 300:
            response = response[:300] + "..."
        
        history_text += f"Assistant: {response}\n\n"
    
    history_text += "\n**Legend:** 🔄 = Query rewritten with history | [Expert 400w] = 400-word Analysis | [Language] = Detected Language"
    
    return history_text

# ===== GRADIO INTERFACE =====

with gr.Blocks(title="Enhanced AgroRAG") as demo:
    gr.Markdown("""
    # 🌾 COMPLETE AgroRAG Assistant (Version1 + Version2 Features)
    
    **All Features Integrated: Expert Analysis • Non-Agri Filtering • History Rewriting • Live Data • AI Intent Detection**
    
    ## 🚀 Version 1 Features (Integrated):
    - 🎓 **Expert Analysis** - EXACTLY 400 words, 800 tokens detailed consultation
    - 🚫 **Non-Agricultural Filtering** - Politely redirects non-farming questions
    - 🔄 **Query Rewriting** - "Tell me more" automatically uses conversation history
    - 🧠 **LLM Intent Analysis** - Detects greetings, agricultural topics, follow-ups
    - 🖼️ **Strict Image Classification** - Only accepts agricultural images
    - 💬 **Greeting Handling** - Friendly responses to "Hi", "Thanks", etc.
    
    ## 🚀 Version 2 Features (Integrated):
    - 📍 **"Here" Location Support** - Set location once, use "here" in all questions
    - 💾 **Session Memory** - Your location saved for the entire session
    - ⚡ **5 Concurrent Users** - Simultaneous query processing
    - 🌦️ **Live Weather Data** - 15-day forecasts from Open-Meteo
    - 🌱 **AI Soil Estimation** - Gemini AI-powered soil analysis
    - 🌾 **Dynamic Crop Database** - Gemini AI-powered requirements
    - 💰 **Crop Price Predictions** - Current prices + 3-month forecasts
    - 🔍 **Smart Follow-ups** - Contextual question suggestions
    - 🌐 **Multilingual** - 13 Indian languages + English
    
    ## 💡 Usage Examples:
    **Basic Questions:**
    - "What is crop rotation?" (gets agricultural answer)
    - "Who won the cricket match?" (politely redirected)
    - "Hello" (friendly greeting response)
    
    **Location-Based:**
    - "I am in Pune" (sets location)
    - "What is the best crop here?" (uses saved location)
    - "Best crops for Mumbai" (gets weather, soil, prices)
    
    **🔄 Conversation History & Query Rewriting:**
    The system automatically detects follow-up questions and rewrites them using conversation history!
    
    **Example 1:**
    - Ask: "What is tomato blight?"
    - Then: "Tell me more about it" → Auto-rewrites to "Tell me more about tomato blight"
    - Then: "How do I prevent this?" → Auto-rewrites to "How do I prevent tomato blight?"
    
    **Example 2:**
    - Ask: "Best fertilizer for rice?"
    - Then: "What about the same for wheat?" → Auto-rewrites with context
    
    **Example 3:**
    - Ask: "How to treat leaf spot disease?"
    - Then: "Continue" → Uses previous topic
    
    **Triggers for rewriting:** "it", "this", "that", "tell me more", "continue", "what about this", "the same"
    
    **Expert Analysis:**
    - Click "Expert Analysis (400w)" for comprehensive 400-word consultation
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📝 Input Your Question")
            
            session_id_input = gr.Textbox(
                label="Session ID (Optional)",
                placeholder="Leave empty for web session",
                visible=True
            )
            
            audio_input = gr.Audio(label="🎤 Voice Input", type="numpy", visible=True)
            
            transcription_output = gr.Textbox(
                label="Transcribed Text",
                lines=2,
                interactive=True,
                placeholder="Voice transcription or type directly..."
            )
            
            text_input = gr.Textbox(
                label="Or Type Your Question",
                lines=3,
                placeholder="Try: 'I am in Pune' then 'What is the best crop here?'"
            )
            
            image_input = gr.Image(label="📸 Upload Image (Optional)", type="pil")
            
            with gr.Row():
                basic_button = gr.Button("Get Answer", variant="primary", size="lg")
                expert_button = gr.Button("🎓 Expert Analysis (400w)", variant="secondary", size="lg")
            
            clear_button = gr.Button("🗑️ Clear All", variant="stop", size="sm")
            
        with gr.Column(scale=2):
            gr.Markdown("### 🤖 Assistant Response")
            
            answer_output = gr.Textbox(
                label="Response",
                lines=25,
                interactive=False,
                show_copy_button=True
            )
            
            gr.Markdown("""
            **📊 Response includes (Version1 + Version2):**
            - **Version1:** LLM intent analysis (agricultural/greeting/follow-up detection)
            - **Version1:** Non-agricultural question filtering
            - **Version1:** Automatic query rewriting with conversation history
            - **Version1:** Expert analysis mode (400 words exactly)
            - **Version2:** Real-time weather forecasts (15 days)
            - **Version2:** AI-estimated soil analysis
            - **Version2:** Dynamic crop suitability scores
            - **Version2:** Crop price predictions (current + 3-month forecast)
            - **Version2:** Contextual follow-up questions
            - **Version2:** Location-specific recommendations
            
            **📋 Response Types:**
            - **Basic Analysis**: Quick, focused answer (150-300 words)
            - **Expert Analysis**: Comprehensive consultation (EXACTLY 400 words, 800 tokens)
            - **Concurrent**: Multiple users can query simultaneously
            - **Thread-Safe**: Isolated session histories
            - **Multilingual**: Auto-translates to Indian languages
            """)
    
    with gr.Accordion("📜 Your Conversation History", open=False):
        with gr.Row():
            refresh_history_btn = gr.Button("🔄 Refresh", variant="secondary")
            clear_history_btn = gr.Button("🗑️ Clear History", variant="secondary")
        
        history_display = gr.Textbox(
            label="Session History", 
            lines=15, 
            interactive=False,
            value="No conversation yet."
        )
    
    with gr.Accordion("⚙️ System Status & API Info", open=False):
        system_status = gr.Textbox(
            label="System Information",
            lines=20,
            interactive=False,
            value=f"""✅ ENHANCED REAL-TIME DYNAMIC AgroRAG System: Active

NEW FEATURES:
- Dynamic Intent Detection: Gemini AI
- "Here" Location Support: Session Memory
- Location Setting: "I am in [City]"
- Max Concurrent: {MAX_CONCURRENT_REQUESTS}
- Live Weather: Open-Meteo API (15 days)
- AI Soil Analysis: Gemini AI
- Dynamic Crops: Gemini AI
- Follow-ups: AI-generated

Cache TTL: Weather 30min, Soil 60min, Crops 120min

API ENDPOINTS:
- fetch_session_id_api()
- answer_question_with_session(q, img, detailed, sid)
- get_conversation_history_by_session(sid)
- clear_conversation_history_by_session(sid)
- get_similar_questions_api(question, location, topic)
- fetch_crop_prices(crop_name, location)

Last Updated: {datetime.now().strftime('%H:%M:%S')}"""
        )
        
        refresh_status_btn = gr.Button("🔄 Refresh Status")
    
    with gr.Accordion("🔑 Session ID Fetcher", open=False):
        fetch_session_btn = gr.Button("Get My Session ID", variant="primary")
        session_id_output = gr.JSON(label="Session ID Response")
    
    # Event Handlers
    audio_input.change(fn=transcribe_only, inputs=[audio_input], outputs=[transcription_output])
    basic_button.click(
        fn=process_basic_question,
        inputs=[transcription_output, text_input, image_input, session_id_input],
        outputs=[answer_output]
    )
    expert_button.click(
        fn=process_expert_question,
        inputs=[transcription_output, text_input, image_input, session_id_input],
        outputs=[answer_output]
    )
    clear_button.click(fn=clear_all_inputs, outputs=[transcription_output, text_input, image_input, answer_output])
    refresh_history_btn.click(fn=refresh_history, outputs=[history_display])
    clear_history_btn.click(fn=clear_conversation_history, outputs=[history_display])
    refresh_status_btn.click(fn=update_system_status, outputs=[system_status])
    fetch_session_btn.click(fn=fetch_session_id_api, outputs=[session_id_output])
    
    # Examples - demonstrating all features including history rewriting
    gr.Examples(
        examples=[
            ["What is the best suitable crop in Pune, Maharashtra?"],  # Version2: Location-based
            ["Which crops can I grow in Varanasi during monsoon season?"],  # Version2: Location + season
            ["What is crop rotation?"],  # Version1: Agricultural question
            ["Who won the cricket match?"],  # Version1: Non-agricultural (will be filtered)
            ["Hello, how can you help me?"],  # Version1: Greeting handling
            ["Thanks for your help!"],  # Version1: Thanks handling
            ["What is tomato blight?"],  # First question for history demo
            ["Tell me more about it"],  # Version1: History rewriting - will use "tomato blight"
            ["How do I prevent this?"],  # Version1: History rewriting - will use previous context
            ["Continue"],  # Version1: History rewriting - continuation
            ["What about the same for potato?"],  # Version1: History rewriting with "the same"
            ["I am in Bangalore"],  # Version2: Location setting
            ["What is the best crop here?"],  # Version2: Using "here" after location set
            ["मुंबई में कौन सी फसल उगाएं?"],  # Multilingual
            ["फसल की यह बीमारी क्या है?"],  # Hindi - agricultural
        ],
        inputs=[text_input],
        label="📚 Example Questions (Testing All Features Including History Rewriting)"
    )

if __name__ == "__main__":
    try:
        demo.launch(
            share=True, 
            server_name="0.0.0.0",
            show_api=True,
            max_threads=MAX_CONCURRENT_REQUESTS * 2
        )
    except OSError as e:
        if "Cannot find empty port" in str(e):
            demo.launch(share=True, show_api=True, max_threads=MAX_CONCURRENT_REQUESTS * 2)
        else:
            raise e

2025-11-04 06:10:16.652155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762236616.895856      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762236616.966122      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Initializing ENHANCED REAL-TIME DYNAMIC AgroRAG System...
Initializing speech recognition...
Initializing Gemini model...
Loading FAISS index...


/tmp/ipykernel_37/1990434726.py:1266: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipykernel_37/1990434726.py:1298: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


✅ ENHANCED REAL-TIME DYNAMIC AgroRAG System Ready!

🚀 **ALL FEATURES FROM VERSION1 + VERSION2:**
✓ LLM-driven agricultural intelligence (version1)
✓ Expert analysis mode (EXACTLY 400 words, 800 tokens) (version1)
✓ Non-agricultural question restriction (version1)
✓ Query rewriting with conversation history (version1)
✓ Stricter agricultural image classification (version1)
✓ Intent analysis with LLM (version1)
✓ Greeting/thanks handling (version1)
✓ Dynamic Intent Detection (Gemini AI) (version2)
✓ "Here" location support with session memory (version2)
✓ Location setting: "I am in [City]" (version2)
✓ Concurrent processing (5 parallel)
✓ Live weather data - 15 days (Open-Meteo API) (version2)
✓ AI-powered soil analysis (Gemini AI) (version2)
✓ Dynamic crop requirements (Gemini AI) (version2)
✓ Crop price prediction with 3-month forecast (Gemini AI) (version2)
✓ Contextual follow-up suggestions (version2)
✓ Similar questions API endpoint (version2)
✓ Location-based recommendations (versi